# Testing classfiers on the 10k pairs dataset

## Loading in data

In [1]:
!pip install pandas 

import pandas as pd


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Load in data

dtype_dict_all = {
    'conversation_id': 'object',
    'id': 'object',
    'author_id': 'object',
    'referenced_tweets_id': 'object',
    'in_reply_to_user_id': 'object',
    'PNR': 'object'
}

pairs = pd.read_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Creating dataset of reference tweets/10k_twitter_pairs.csv', dtype=dtype_dict_all )

pairs

,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,A/R,fasttext_cos_sim_hate_sentence,fasttext_cos_sim_prosocial_sentence,tweeter_username,tweeter_name,pair_num,type,like_n,retweet_n,quote_n
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,0-0,0.762309,0.959726,NaN,NaN,1,reply,1,0,0
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,NaN,NaN,NaN,frkomo,Sarah Agerklint,1,tweet,1,0,0
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,0-0,0.836744,0.958750,NaN,NaN,2,reply,0,0,0
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,NaN,NaN,NaN,PeterHuggler,Peter Huggler,2,tweet,1,0,0
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,0-0,0.746267,0.957824,NaN,NaN,3,reply,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1430605648878784513,da,2021-08-25 19:00:13,1430605952722604045,@PSkipperEL Træk den røde linje nu.\n\n#VæltMe...,2345329126,NaN,NaN,611076925,NaN,...,NaN,NaN,NaN,M_K_Petersen,Michael K. Petersen,9998,tweet,16,0,0
19996,822180433681088513,da,2017-01-19 20:55:03,822185628620226560,@sllaursen har svært ved at afgøre om jeg ægte...,546738902,1.0,822180433681088513,43529378,1007802400,...,0-0,0.886249,0.839126,NaN,NaN,9999,reply,3,0,0
19997,822180433681088513,da,2017-01-19 20:34:25,822180433681088513,"Når et sponsorat rammer ESPN, er man kommet ok...",43529378,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,steen,Steen Laursen,9999,tweet,13,1,0
19998,953648851721826304,da,2018-01-17 18:22:52,953694124049420288,@emilnielsen @pelledragsted Tak skal du have. ...,26210681,0.0,953655858675757057,19391295,2010772527,...,0-1,0.886244,0.897734,NaN,NaN,10000,reply,10,0,0


In [3]:
pairs.dtypes

conversation_id                         object
lang                                    object
created_at                              object
id                                      object
text                                    object
author_id                               object
replied_to_reply_count                 float64
referenced_tweets_id                    object
in_reply_to_user_id                     object
PNR                                     object
surveyXact_externke                     object
non_unique_twitter_author_id           float64
started_survey                         float64
rec-nition                             float64
attack                                 float64
A/R                                     object
fasttext_cos_sim_hate_sentence         float64
fasttext_cos_sim_prosocial_sentence    float64
tweeter_username                        object
tweeter_name                            object
pair_num                                 int64
type         

## Zero shot learning

In zero shot learning a pre-trained model is used to predict the give a probability on any given label using embeddings, even though the label(s) has not been part of the training data

### facebook/bart-large-mnli model

In [4]:
!pip install --upgrade transformers
from transformers import pipeline


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
pipe_1 = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [6]:
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

sequence_to_classify_1 = "i see your point but i dont think it is very constructive"
sequence_to_classify_2 = "jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv"
sequence_to_classify_3 = pairs['text'][3]
sequence_to_classify_4 = "today i went for a walk outside with my dog, nice sunny weather"

pipe_1(sequence_to_classify_2, candidate_labels)


{'sequence': 'jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv',
 'labels': ['counterspeech', 'neutral tone', 'hate'],
 'scores': [0.819023072719574, 0.17399725317955017, 0.006979694124311209]}

In [7]:
# on test data

from tqdm import tqdm
pairs_test= pairs.head(300)

# Candidate labels
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

# List to store the scores
counterspeech_scores = []
hate_scores = []
neutral_scores = []

# Iterate over each text
for text in (tqdm(pairs_test['text'], desc= "Extracting scores")) :
    result = pipe_1(text, candidate_labels)
    # Extract the scores
    counterspeech_score = dict(zip(result['labels'], result['scores']))['counterspeech']
    hate_score = dict(zip(result['labels'], result['scores']))['hate']
    neutral_score = dict(zip(result['labels'], result['scores']))['neutral tone']
    # Append the scores to the lists
    counterspeech_scores.append(counterspeech_score)
    hate_scores.append(hate_score)
    neutral_scores.append(neutral_score)

# Add the scores as a new column in the DataFrame
pairs_test['counterspeech_score'] = counterspeech_scores
pairs_test['hate_score'] = hate_scores
pairs_test['neutral_score'] = neutral_scores


# Initialize the 'counterspeech' column with 'NA'
pairs_test['counterspeech'] = 'NA'

# Update the 'counterspeech' column based on the 'counterspeech_score'
for index, row in pairs_test.iterrows():
    if row['counterspeech_score'] > 0.8:
        pairs_test.at[index, 'counterspeech'] = 'yes'
    else:
        pairs_test.at[index, 'counterspeech'] = 'no'    


pairs_test

Extracting scores: 100%|██████████████████████| 300/300 [01:36<00:00,  3.09it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/2575739544.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['counterspeech_score'] = counterspeech_scores
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/2575739544.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['hate_score'] = hate_scores
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/2575739544.p

,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,tweeter_name,pair_num,type,like_n,retweet_n,quote_n,counterspeech_score,hate_score,neutral_score,counterspeech
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,NaN,1,reply,1,0,0,0.765376,0.008536,0.226088,no
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,Sarah Agerklint,1,tweet,1,0,0,0.853204,0.006963,0.139833,yes
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,NaN,2,reply,0,0,0,0.575550,0.030307,0.394142,no
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,Peter Huggler,2,tweet,1,0,0,0.616760,0.015232,0.368008,no
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,NaN,3,reply,1,0,0,0.815085,0.017978,0.166937,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1231494237130280966,da,2020-02-23 08:50:33,1231501587534827520,@SteffenFrolund @MonbergSF @jonasholmdk Og præ...,97198778,NaN,NaN,33859175,NaN,...,Lars Køhler,148,tweet,5,0,0,0.608624,0.046957,0.344419,no
296,1414594602275901443,da,2021-07-12 14:39:33,1414595287532834818,"@MVangby Er det for grundspillet, eller hele s...",805874425988087811,2.0,1414594602275901443,835144095165202433,NaN,...,NaN,149,reply,1,0,0,0.812016,0.027733,0.160252,yes
297,1414594602275901443,da,2021-07-12 14:36:49,1414594602275901443,"Min Superliga forudsigelse, her 1,5 måned inde...",835144095165202433,NaN,NaN,NaN,NaN,...,Mikkel Vangby,149,tweet,37,0,0,0.868525,0.042197,0.089278,yes
298,1198177606077734913,da,2019-11-23 10:25:50,1198185883821846530,@steenHiversen @R4nd4hl @uffeelbaek @naserkhad...,26210681,2.0,1198185017312186368,2554415384,2010772527,...,NaN,150,reply,4,0,0,0.710413,0.024398,0.265189,no


In [44]:
#On full dataset

from tqdm import tqdm

# Candidate labels
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

# List to store the scores
counterspeech_scores = []
hate_scores = []
neutral_scores = []

# Iterate over each text
for text in (tqdm(pairs['text'], desc= "Extracting scores")) :
    result = pipe_1(text, candidate_labels)
    # Extract the scores
    counterspeech_score = dict(zip(result['labels'], result['scores']))['counterspeech']
    hate_score = dict(zip(result['labels'], result['scores']))['hate']
    neutral_score = dict(zip(result['labels'], result['scores']))['neutral tone']
    # Append the scores to the lists
    counterspeech_scores.append(counterspeech_score)
    hate_scores.append(hate_score)
    neutral_scores.append(neutral_score)

# Add the scores as a new column in the DataFrame
pairs['counterspeech_score'] = counterspeech_scores
pairs['hate_score'] = hate_scores
pairs['neutral_score'] = neutral_scores


# Initialize the 'counterspeech' column with 'NA'
pairs['counterspeech'] = 'NA'

# Update the 'counterspeech' column based on the 'counterspeech_score'
for index, row in pairs.iterrows():
    if row['counterspeech_score'] > 0.8:
        pairs.at[index, 'counterspeech'] = 'yes'
    else:
        pairs.at[index, 'counterspeech'] = 'no'    


pairs

Extracting scores:   4%|▋                 | 797/20000 [07:40<3:05:01,  1.73it/s]


KeyboardInterrupt: 

### cross-encoder/nli-deberta-base model

In [8]:
pipe_2 = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-base", device=0)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

sequence_to_classify_1 = "i see your point but i dont think it is very constructive"
sequence_to_classify_2 = "jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv"


pipe_2(sequence_to_classify_1, candidate_labels)

#doesnt seem very good

{'sequence': 'i see your point but i dont think it is very constructive',
 'labels': ['counterspeech', 'neutral tone', 'hate'],
 'scores': [0.5037503242492676, 0.4367963671684265, 0.05945330858230591]}

### TarsClassifier 

In [11]:
!pip install Flair


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
from flair.models import TARSClassifier
from flair.data import Sentence


tars = TARSClassifier.load('tars-base')


sentence = Sentence("jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv")
sentence_2 = Sentence("i understand, but i dont agree")


classes = ["counterspeech", "hate"]

tars.predict_zero_shot(sentence_2, classes)

print(sentence_2)

2024-07-31 11:17:34,459 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model
2024-07-31 11:17:34,571 Provided `None` does not exist in the model. Consider calling `add_and_switch_to_new_task` first.
2024-07-31 11:17:34,571 `ZeroShot` is the current task. Switch to some other task before dropping this.
Sentence[7]: "i understand, but i dont agree"


## Applying classification models

#### Counterargument classifier (from huggingface)


https://huggingface.co/ThinkCERCA/counterargument_hugging

In [13]:
pipe_3 = pipeline("text-classification", model="ThinkCERCA/counterargument_hugging", device=0)

In [34]:
# Gives labels: Not Counterargument or Counterargument with a score

text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "your a fool"
text_4 = "i like dogs, but also cats"

# Apply the pipeline to the text
result = pipe_3(text_4)
result

#allright?
#maybe test this on the test df

[{'label': 'Not Counterargument', 'score': 0.6886796355247498}]

In [47]:
# Test on small df

# List to store the scores
labels = []
scores = []

# Iterate over each text
for text in (tqdm(pairs_test['text'], desc= "Extracting scores")) :
    result = pipe_3(text)
    # Extract the scores
    label = result[0]['label']
    score = result[0]['score']
    # Append the scores to the lists
    labels.append(label)
    scores.append(score)

# Add the scores as a new column in the DataFrame
pairs_test['label'] = labels
pairs_test['score'] = scores


# Initialize the 'counterspeech' column with 'NA'
#pairs_test['counterspeech'] = 'NA'

# Update the 'counterspeech' column based on the 'counterspeech_score'
#for index, row in pairs_test.iterrows():
    #if row['counterspeech_score'] > 0.8:
        #pairs_test.at[index, 'counterspeech'] = 'yes'
    #else:
        #pairs_test.at[index, 'counterspeech'] = 'no'    


pairs_test

Extracting scores: 100%|██████████████████████| 300/300 [00:42<00:00,  7.03it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/454167464.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['label'] = labels
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/454167464.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['score'] = scores


,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,type,like_n,retweet_n,quote_n,counterspeech_score,hate_score,neutral_score,counterspeech,label,score
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,reply,1,0,0,0.765376,0.008536,0.226088,no,Not Counterargument,0.635118
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,tweet,1,0,0,0.853204,0.006963,0.139833,yes,Counterargument,0.503948
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,reply,0,0,0,0.575550,0.030307,0.394142,no,Not Counterargument,0.648885
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,tweet,1,0,0,0.616760,0.015232,0.368008,no,Not Counterargument,0.664990
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,reply,1,0,0,0.815085,0.017978,0.166937,yes,Not Counterargument,0.723273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1231494237130280966,da,2020-02-23 08:50:33,1231501587534827520,@SteffenFrolund @MonbergSF @jonasholmdk Og præ...,97198778,NaN,NaN,33859175,NaN,...,tweet,5,0,0,0.608624,0.046957,0.344419,no,Not Counterargument,0.813830
296,1414594602275901443,da,2021-07-12 14:39:33,1414595287532834818,"@MVangby Er det for grundspillet, eller hele s...",805874425988087811,2.0,1414594602275901443,835144095165202433,NaN,...,reply,1,0,0,0.812016,0.027733,0.160252,yes,Not Counterargument,0.805045
297,1414594602275901443,da,2021-07-12 14:36:49,1414594602275901443,"Min Superliga forudsigelse, her 1,5 måned inde...",835144095165202433,NaN,NaN,NaN,NaN,...,tweet,37,0,0,0.868525,0.042197,0.089278,yes,Not Counterargument,0.837815
298,1198177606077734913,da,2019-11-23 10:25:50,1198185883821846530,@steenHiversen @R4nd4hl @uffeelbaek @naserkhad...,26210681,2.0,1198185017312186368,2554415384,2010772527,...,reply,4,0,0,0.710413,0.024398,0.265189,no,Not Counterargument,0.751263


In [51]:
#check value counts
pairs_test['label'].value_counts(dropna=False) 


label
Not Counterargument    235
Counterargument         65
Name: count, dtype: int64

In [50]:
# look at performance

#pull up text examples
Counterargument = pairs_test[pairs_test['label'] == 'Counterargument']
no_Counterargument = pairs_test[pairs_test['label'] == 'Not Counterargument']


print(Counterargument['text'][:100])

# can print the whole text using
text_to_print = Counterargument['text'].head(20).tolist()
text_to_print

text_to_print = no_Counterargument['text'].head(20).tolist()
text_to_print

#seems pretty good, not perfect though 


1      @MonbergSF Sig det til spillerforeningen, som ...
8      @larskohler @BEsbensen @PiaOlsen Lidt, men jeg...
12     @quitte74 @BrondbyIF Skal jeg tage din med, nå...
14     @HellaSchulin @dksvin @KopenhagenFurDK @spisek...
17     @KWEkristian Ah, det er da et lidt tyndt svar,...
                             ...                        
276    @BrianMork @Noerloev @MagnusPharao @stinelinne...
281    @MagnusPharao Min oplevelse er at disse sensat...
285    @MonbergSF @PEF40 Øhhh derfor jeg spørger noge...
293    Influencerne tror de har opfundet flow-TV http...
299    @ammitzbollbille @R4nd4hl @uffeelbaek @naserkh...
Name: text, Length: 65, dtype: object


['@frkomo Jeg siger det vel strengt taget bare til dem der følger mig ;) Men hvis du følger den efterfølgende diskussion, så er jeg netop ikke imponeret over den tilgang spillerne har:)',
 '@PeterHuggler Had alt det, du vil. Men du skal ikke gøre dig til dommer over, hvad jeg mener er en naturlig reaktion. Jeg dækker overhovedet ikke den sag.',
 '@brianweichardt Jeg hader den her slags: Du går ind og gør dig selv til dommer, udfra et foto. Lad retten gøre sit arbejde, og søg alene at formidle sagen.',
 '@nielscallesoe @Heunicke Din første indvending er muligvis korrekt. Det skal vurderes. Den anden forstår jeg ikke. Hvilke andre vacciner tilbyder vi off-label?',
 '@stinuslindgreen @Heunicke I min optik, er det ikke en relevant bekymring. Flere grunde. Vigtigste: For det første, fordi vi ikke har skala til at gøre en forskel. HVIS det giver øget selektion i den retning, sker det uanset hvad vi gør. For det andet, undlader vi ikke andre vacciner med den begrundelse.',
 '@RasmusMalver @ra

In [69]:
# Sort by 'score' in descending order
sorted_df = Counterargument.sort_values('score', ascending=False)

#print 
sorted_df['text'].head(20).tolist()
#sorted_df[['text', 'score', 'label']].head(10)

['@quitte74 @pferdinandsen Tusind tak, så har vi lidt at kigge på. Jeg ville selv have elsket de samme muligheder, men Basic på en Commodore 64 var nu heller ikke af vejen😉',
 '@KWEkristian Ah, det er da et lidt tyndt svar, når I selv har startet debatten - som du ved, har vi bestyrelsesmøde, så der bliver vist ikke megen regional deltagelse i den første del af jeres møde. @SigneLoentoft',
 '@quitte74 @PabloBresciani @ChrRothmann @stentoft01 @gratisbilletter @billetboers Jeg er vild med det man kan med kode, men har aldrig selv interesseret mig for hvordan det skal gøres. Så “vores” udvikler har lært at han enten skal sige “fint, det gør vi” eller “det kan du ikke få”, når jeg ønsker mig nye funktioner. Vi taler ikke længere om hvorfor 😂',
 '@Sklander89 @quitte74 Sklander for pokker. Du var selv direkte rasende over episoden, og nu beskylder du simpelthen mig for at lyve om det... 🤠 https://t.co/7GjV1bOVVO',
 '@JonasKGaba @alternativet_ @uffeelbaek Af de 7 påstande, vil jeg sige at 5 o

#### Bert counterspeech classifier (from huggingface)

Bert finetuned on the CONAN dataset

https://huggingface.co/tum-nlp/bert-counterspeech-classifier

In [22]:
pipe_4 = pipeline("text-classification", model="tum-nlp/bert-counterspeech-classifier",device=0)

In [27]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you are a fool"

# Apply the pipeline to the text
result = pipe_4(text_3)
result

#Seems to work well on english data, not so good on danish data..

[{'label': 'counter-argument', 'score': 0.8415255546569824}]

#### Counterspeech score classifier (from huggingface)

https://huggingface.co/Hate-speech-CNERG/new-counterspeech-score

In [28]:
pipe_5 = pipeline("text-classification", model="Hate-speech-CNERG/new-counterspeech-score", device=0)

In [29]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you are a fool"

# Apply the pipeline to the text
result = pipe_5(text_3)
result

# What labels, and what are the scores, test

[{'label': 'NEGATIVE', 'score': 0.7565722465515137}]

In [70]:
# Test on small df

# List to store the scores
labels_counter = []
scores_counter = []

# Iterate over each text
for text in (tqdm(pairs_test['text'], desc= "Extracting scores")) :
    result = pipe_5(text)
    # Extract the scores
    label = result[0]['label']
    score = result[0]['score']
    # Append the scores to the lists
    labels_counter.append(label)
    scores_counter.append(score)

# Add the scores as a new column in the DataFrame
pairs_test['label_counter'] = labels_counter
pairs_test['score_counter'] = scores_counter

Extracting scores: 100%|██████████████████████| 300/300 [00:43<00:00,  6.84it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/1298344427.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['label_counter'] = labels_counter
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_15413/1298344427.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['score_counter'] = scores_counter


In [71]:
pairs_test['label_counter'].value_counts(dropna=False) 

label_counter
POSITIVE    180
NEGATIVE    120
Name: count, dtype: int64

In [77]:
positive = pairs_test[pairs_test['label_counter'] == 'POSITIVE']


# Sort by 'score' in descending order
sorted_df = positive.sort_values('score', ascending=False)

#print 
sorted_df['text'].head(20).tolist()
#sorted_df[['text', 'score_counter', 'label_counter']].head(10)

['Min mors æblekage. Vi har prøvet at genlave den efter hun gik bort. Vi har hendes nedskrevne opskrift, men det smager bare ikke ens. Som hun nærmest har taget den med sig. Har I også sådan nogle retter/opskrifter i jeres familie?',
 '@olufsphere @AarhusGinger Vi som forældre har lidt hånd i hanke med hvad der er lavet / ikke er lavet + det samme med skolen. Det ser jeg som en ganske naturlig ting. \n\nJeg ved ikke hvorfor du vil blande den lockout ind i det her. jeg forholder mig udelukkende til den nuværende situation.',
 'For præcis elleve år siden kom jeg til København som håbefuld studerende (og aspirerende fodboldtræner).\n\nMeget er sket siden, men når jeg ser det her billede fra dengang, tænker jeg stadig:\n\n"Tænk, at en tysker kan have så meget charme, selvtillid og lækkert hår." https://t.co/aljCMBxZgu',
 '@PiaOlsen det respektere jeg 100%. Flot du tager dig tid til dialogen, politisk bliver vi nok ikke enige,men respekt for politisk engagement',
 '@lauralindahl Det er det 

## Applying hate classification model(s)

### HateXplain

In [ ]:
#Gives hate, offensive and normal 

In [87]:
# Use a pipeline as a high-level helper
pipe_6 = pipeline("text-classification", model="Hate-speech-CNERG/bert-base-uncased-hatexplain", device= 0)


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [100]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you cute dog"
text_4 = 'du er så tyk og grim'

# Apply the pipeline to the text
result = pipe_6(text_4)
result

#not so good on danish data?
#test

[{'label': 'normal', 'score': 0.7060950398445129}]

### Hate Bert

In [78]:
pipe_7 = pipeline("fill-mask", model="GroNLP/hateBERT", device= 0)

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [86]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT")

# Example text
text = "i do not hate you"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted probabilities
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Print the results
print(f"Probabilities: {probabilities}")

# Get the predicted label
predicted_label = torch.argmax(probabilities, dim=1).item()
print(f"Predicted label: {predicted_label}")


Probabilities: tensor([[0.4175, 0.5825]])
Predicted label: 1


In [79]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you are a fool"

# Apply the pipeline to the text
result = pipe_6(text_3)
result

PipelineException: No mask_token ([MASK]) found on the input

## Finetuning a model on labeled counterspeech data (CONAN)

CONAN (COunter NArratives through Nichesourcing): A dataset containing counterspeech responses to hate speech. Data is French, italian and English

https://github.com/marcoguerini/CONAN

https://aclanthology.org/P19-1271.pdf

## Applying the jigsaw perspective API
